In [3]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from skimage.transform import resize
from tensorflow.keras import datasets, layers, models

## Simple CNN

In [4]:
resize_factor = (188, 250)

train_img_path = '../data/train_set/train_images/'
test_img_path = '../data/train_set/test_images/'
train_features_path = '../data/train_set/train_points/'
test_features_path = '../data/train_set/test_points/'
labels = pd.read_csv('../data/train_set/label.csv')

train_img = sorted(os.listdir(train_img_path))
test_img = sorted(os.listdir(test_img_path))

In [5]:
### Train emotion indexes
train_indexes = []
for img in train_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    train_indexes.append(img - 1)
    
emotion_idx = labels[['emotion_idx']].loc[train_indexes].values[:, 0] - 1

### Test emotion indexes
test_indexes = []
for img in test_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    test_indexes.append(img - 1)
    
test_emotion_idx = labels[['emotion_idx']].loc[test_indexes].values[:, 0] - 1  

In [14]:
imgs = []
start_time = time.time()
for img in train_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join(train_img_path, img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    imgs.append(img)
print("--- %s seconds ---" % (time.time() - start_time))  

imgs_test = []
start_time = time.time()
for img in test_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join('../data/train_set/test_images/', img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    imgs_test.append(img)
print("--- %s seconds ---" % (time.time() - start_time)) 

--- 77.71262311935425 seconds ---
--- 20.534557104110718 seconds ---


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(188, 250, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(22, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(np.array(imgs), emotion_idx, epochs=10, validation_data=(np.array(imgs_test), test_emotion_idx))

Train on 1999 samples, validate on 501 samples
Epoch 1/10
1999/1999 [==============================] - 481s 241ms/step - loss: 3.1356 - acc: 0.0445 - val_loss: 3.0912 - val_acc: 0.0579
Epoch 2/10
1999/1999 [==============================] - 490s 245ms/step - loss: 3.0909 - acc: 0.0415 - val_loss: 3.0911 - val_acc: 0.0559
Epoch 3/10
 768/1999 [==========>...................] - ETA: 4:32 - loss: 3.0926 - acc: 0.0573

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-26618e42ffc2>", line 13, in <module>
    history = model.fit(np.array(imgs), emotion_idx, epochs=10, validation_data=(np.array(imgs_test), test_emotion_idx))
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1363, in fit
    validation_steps=validation_steps)
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_arrays.py", line 264, in fit_loop
    outs = f(ins_batch)
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/backend.py", line 2914, in __call__
    fetched = self._callable_fn(*array_vals)
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1382, in __call__
    run_metadata_ptr)
KeyboardInterrupt

During handling of the abov

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 432, in _run_callback
    callback(*args, **kwargs)
  File "/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 276, in null_wrapper
    return fn(*args, **kwargs)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 642, in _abort_queue
    ide

## CNN with Xception pretrained model:
In this model we use a pretrained model to find the fteatures, and then add a 3 layer CNN on top of that model as a classifier

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D
import tensorflow.contrib.eager as tfe
from matplotlib import pyplot as plt
import numpy as np
import os

tf.enable_eager_execution()

/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [6]:
class XceptionBottleneck(tf.keras.Model):
    
    def __init__(self):
        super(XceptionBottleneck, self).__init__()
        
        # Define xception layer (include_top=False and use imagenet weights), 
        # see: https://keras.io/applications/#models-for-image-classification-with-weights-trained-on-imagenet
        self.xception_layers = Xception(weights='imagenet', include_top=False)
        
        # Define pooling GlobalAveragePooling2D 
        # see: https://keras.io/layers/pooling/  
        self.pooling_layer = GlobalAveragePooling2D()
        
    def call(self, inputs):
        result = self.xception_layers(inputs)
        result = self.pooling_layer(result)
        return result

In [7]:
def create_dataset_fmr_images(file_paths):
    # Reads an image from a file, decodes it into a dense tensor, and resizes it
    # to a fixed shape. Read more here: https://www.tensorflow.org/guide/datasets#decoding_image_data_and_resizing_it
    def _parse_function(filename):
        image_string = tf.read_file(filename)
        image_decoded = tf.image.decode_png(image_string)
        image_resized = tf.image.resize_images(image_decoded, [200, 200])
        return image_resized

    file_paths = tf.constant(file_paths)
    
    dataset = tf.data.Dataset.from_tensor_slices((file_paths))
    dataset = dataset.map(_parse_function)

    return dataset

def cache_bottleneck_layers(file_paths, batch_size, device):
    
    bottle_necks = []
    dataset = create_dataset_fmr_images(file_paths).batch(batch_size)
    n_samples = len(file_paths)

    device = "gpu:0" if tfe.num_gpus() else "cpu:0"
    
    with tf.device(device):
        xception_out = XceptionBottleneck()
        for batch_num, image in enumerate(dataset):
            print('\rComputing bottle neck layers... batch {} of {}'.format(batch_num+1, n_samples//batch_size), end="")
            
            # Compute bottle necks layer for image batch convert to numpy and append to bottle_necks
            # ...
            # ...
            result = xception_out(image)
            result = result.numpy()
            bottle_necks.append(result)
            
    return np.vstack(bottle_necks)

In [ ]:
train_img_path = '../data/train_set/train_images/'
train_img = sorted(os.listdir(train_img_path))
train_indexes = []
for img in train_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    train_indexes.append(img - 1)
    
emotion_idx = labels[['emotion_idx']].loc[train_indexes].values[:, 0] - 1

In [8]:
train_file_paths = []
for img in train_img:
    if img == ".DS_Store":
        continue
    train_file_paths.append(os.path.join(train_img_path, img))

In [9]:
start_time = time.time()

device = "gpu:0" if tfe.num_gpus() else "cpu:0"
bottle_necks = cache_bottleneck_layers(train_file_paths, batch_size=50, device=device)

print("--- %s seconds ---" % (time.time() - start_time)) 

Computing bottle neck layers... batch 40 of 39--- 864.5508890151978 seconds ---


In [10]:
cache_path = "./cache/"
fname = 'bottle_neck.npz'
save_path = os.path.join(cache_path,fname)

if not os.path.isdir(cache_path): 
    os.mkdir(cache_path)
    
np.savez(save_path, bottle_necks=bottle_necks, labels=emotion_idx)

In [31]:
class XceptionClassifier(tf.keras.Model):
    
    def __init__(self, n_classes):
        super(XceptionClassifier, self).__init__()
        # Define the layer(s) you would like to use for your classifier
        self.dense_layer_1 = tf.keras.layers.Dense(units=20*n_classes, activation=tf.keras.activations.relu)
        self.dense_layer_2 = tf.keras.layers.Dense(units=5*n_classes, activation=tf.keras.activations.relu)
        self.dense_layer_3 = tf.keras.layers.Dense(units=n_classes)
        
    def call(self, inputs):
        # Set this up appropriately, will depend on how many layers you choose
        result = self.dense_layer_1(inputs)
        result = self.dense_layer_2(result)
        result = self.dense_layer_3(result)
        return result

In [32]:
# run this once for all models, in order to have same shuffeled data_set for all of them.
data = np.load(save_path)
train_bottle_necks, train_labels = data['bottle_necks'],  data['labels']
n_train_samples= len(train_bottle_necks)

# zip the index of images with their indexes
train_bottle_necks = list(zip(range(n_train_samples), train_bottle_necks))

# splitting the data to three different part. 70% for train the model, 15% for the validation modeling, and 15% for
# trian modeling

train_bottle_necks, X_test, train_labels, Y_test = train_test_split(train_bottle_necks, train_labels, test_size=0.3, random_state=40)
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.5, random_state=40)

# unzip the train_bottle_necks
_, train_bottle_necks = zip(*train_bottle_necks)
train_bottle_necks = np.array(train_bottle_necks)

_, X_val = zip(*X_val)
X_val = np.array(X_val)

index_test, X_test = zip(*X_test)
X_test = np.array(X_test)

In [38]:
batch_size = 16        # You will play around with this 
n_epochs = 200         # Choose num epochs based on when you think the parameters have converged
learning_rate = 0.00001 # You will try different learning rates

train_loss_history = []

In [39]:
train_images_dataset = tf.data.Dataset.from_tensor_slices(train_bottle_necks)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(train_labels)
train_dataset = tf.data.Dataset.zip((train_images_dataset, train_labels_dataset))
train_dataset = train_dataset.batch(batch_size) 
# since in train_test split the shuffle is True, and having the same train data set for all models, I do not call
# 'shuffle(buffer_size=n_train_samples)' here.

In [40]:
x_classifier = XceptionClassifier(n_classes=22)
optimizer = tf.train.AdamOptimizer(learning_rate) 

In [41]:
start_time = time.time()
with tf.device(device):
    for epoch in range(n_epochs):
        for batch, (images, labels) in enumerate(train_dataset):

            with tf.GradientTape() as tape:
                # Compute logits
                logits = x_classifier(images)
                xe_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
                
            train_loss_history.append(xe_loss.numpy())
            # Compute gradient and apply gradients
            grads = tape.gradient(xe_loss, x_classifier.variables)
            optimizer.apply_gradients(zip(grads, x_classifier.variables),
                                      global_step=tf.train.get_or_create_global_step())
            
            if batch % 10 == 0:
                print('\rEpoch: {}, Batch: {}, Loss: {}'.format(epoch, batch, train_loss_history[-1]), end='')
print("--- %s seconds ---" % (time.time() - start_time)) 

Epoch: 199, Batch: 80, Loss: 1.84995436668396--- 96.57389521598816 seconds ---


In [42]:
## computing the train accuracy for first model
logits = x_classifier(tf.constant(train_bottle_necks))
y_pred = tf.nn.softmax(logits).numpy()
train_pred = np.argmax(y_pred, axis=1)
train_accuracy = np.sum(train_pred == train_labels) / len(train_pred)

## computing the validation accuracy for first model
logits = x_classifier(tf.constant(X_val))
y_pred = tf.nn.softmax(logits).numpy()
val_pred = np.argmax(y_pred, axis=1)
val_accuracy = np.sum(val_pred == Y_val) / len(val_pred)

print(train_accuracy, val_accuracy)
# .... complete parts (e,f)

0.3909935668334525 0.043333333333333335
